In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import string
import os
warnings.filterwarnings('ignore')

# 원본 날씨 데이터 가져오기

In [2]:
year=input('정리할 년도를 입력하세요 : ')

정리할 년도를 입력하세요 : 2020


In [3]:
path_dir = "data/원본_초미세먼지/"
file_list = os.listdir(path_dir)
file_list = [x for x in file_list if year in x]
length=len(file_list)
for i in range(1,length+1):
    globals()["weather_{}".format(i)] = pd.read_csv(path_dir+"{}_{}.csv".format(year,i), encoding='cp949')
    print("complete load : {}년 {}월".format(year,i))

complete load : 2020년 1월
complete load : 2020년 2월
complete load : 2020년 3월
complete load : 2020년 4월
complete load : 2020년 5월
complete load : 2020년 6월
complete load : 2020년 7월
complete load : 2020년 8월
complete load : 2020년 9월
complete load : 2020년 10월
complete load : 2020년 11월
complete load : 2020년 12월


In [4]:
gu_ko=['서울시 평균','강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', 
       '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', 
       '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', 
       '은평구', '종로구', '중구', '중랑구']

# 행 열 전환

In [5]:
for i in range(1,length+1):
    for j in range(31,1,-1):
        if globals()["weather_{}".format(i)]["{}일".format(j)].isnull().sum()==len(globals()["weather_{}".format(i)]):
            globals()["weather_{}".format(i)].drop(["{}일".format(j)], axis=1, inplace=True)
        else:
            break

In [6]:
for i in range(1,length+1):
    globals()["weather_{}".format(i)]=globals()["weather_{}".format(i)].transpose()
    globals()["weather_{}".format(i)].rename(columns=globals()["weather_{}".format(i)].iloc[0],inplace=True)
    globals()["weather_{}".format(i)] = globals()["weather_{}".format(i)].drop(globals()["weather_{}".format(i)].index[0])
    globals()["weather_{}".format(i)].reset_index(drop=True, inplace=True)

# 년 월 일 추가

In [7]:
for i in range(1,length+1):
    globals()["weather_{}".format(i)]['년']=year
    globals()["weather_{}".format(i)]['월']=i
    globals()["weather_{}".format(i)]['일']=range(1,len(globals()["weather_{}".format(i)])+1)

In [8]:
result=pd.DataFrame(columns=['년','월','일','초미세먼지','자치구'])

In [9]:
for i in range(1,length+1):
    average=globals()["weather_{}".format(i)]['서울시 평균'].to_list()
    for j in range(0,len(gu_ko)):
        temp=globals()["weather_{}".format(i)][['년','월','일',gu_ko[j]]]
        temp=temp.rename(columns={gu_ko[j]:"초미세먼지"})
        temp['초미세먼지'].fillna(value=pd.Series(average),inplace=True) #결측치 서울시 평균으로 채우기
        temp['자치구']=gu_ko[j]
        result=result.append(temp)
        del temp
result.reset_index(drop=True, inplace=True)
result

,년,월,일,초미세먼지,자치구
0,2020,1,1,23.0,서울시 평균
1,2020,1,2,40.0,서울시 평균
2,2020,1,3,47.0,서울시 평균
3,2020,1,4,38.0,서울시 평균
4,2020,1,5,36.0,서울시 평균
...,...,...,...,...,...
9511,2020,12,27,36.0,중랑구
9512,2020,12,28,40.0,중랑구
9513,2020,12,29,37.0,중랑구
9514,2020,12,30,13.0,중랑구


# 등급 나누기

In [10]:
result['초미세먼지'].isnull().sum()

0

In [11]:
result['초미세먼지']=result['초미세먼지'].astype(float)

In [12]:
result.loc[(result.초미세먼지>=0) & (result.초미세먼지<=15),'등급']='좋음'
result.loc[(result.초미세먼지>15) & (result.초미세먼지<=35),'등급']='보통'
result.loc[(result.초미세먼지>35) & (result.초미세먼지<=75),'등급']='나쁨'
result.loc[(result.초미세먼지>75),'등급']='매우나쁨'

In [13]:
result.drop(['초미세먼지'], axis=1, inplace=True)
result.rename(columns={"등급":"초미세먼지"},inplace=True)
result

,년,월,일,자치구,초미세먼지
0,2020,1,1,서울시 평균,보통
1,2020,1,2,서울시 평균,나쁨
2,2020,1,3,서울시 평균,나쁨
3,2020,1,4,서울시 평균,나쁨
4,2020,1,5,서울시 평균,나쁨
...,...,...,...,...,...
9511,2020,12,27,중랑구,나쁨
9512,2020,12,28,중랑구,나쁨
9513,2020,12,29,중랑구,나쁨
9514,2020,12,30,중랑구,좋음


In [14]:
store_path_dir = "data/초미세먼지"
if os.path.isdir(store_path_dir)==False:
    os.makedirs(store_path_dir)
result.to_csv("{}/{}_초미세먼지.csv".format(store_path_dir,year),index=False, encoding='cp949')

In [15]:
result.isnull().sum()

년        0
월        0
일        0
자치구      0
초미세먼지    0
dtype: int64